In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothic'


import os, sys, random
from pathlib import Path
import glob

from tqdm import tqdm
from fastprogress import progress_bar

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
CUR_DIR = Path.cwd()
DATA_DIR = CUR_DIR / 'data/preprocessed'
DATA_DIR

WindowsPath('C:/Users/안현준/풍년보험/data/preprocessed')

In [3]:
crops = ['감자', '고구마', '고추(화건)', '깻잎', '녹두(국산)', '단감', '당근(무세척)', '대파', '들깨(국산)',
       '딸기', '무', '미나리', '방울토마토', '배(신고)', '배추', '백참깨(국산)', '복숭아(백도)',
       '사과(후지)', '생강(국산)', '수박', '시금치', '애호박', '양배추', '양파', '오이(가시)', '적상추',
       '참외', '콩(흰콩국산)', '토마토', '팥(국산)', '포도(캠벨)', '풋고추']
TARGET_YEAR = [2017, 2018, 2019, 2020]

# 연도별 가격 집계

In [5]:
from collections import defaultdict
insurance = defaultdict(dict)

for crop in progress_bar(crops):
    path = f'./results/{crop}/predicts_{crop}.csv'
    df = pd.read_csv(path, encoding='utf-8')
    df['year'] = df['연월'].apply(lambda x: int(str(x[:4])))
    df['month'] = df['연월'].apply(lambda x: int(str(x[5:])))
    
    # 연도별 가격 평균 집계
    df_group = df.groupby(['year']).agg({f'{crop}_true': 'mean', f'{crop}_pred': 'mean'}).reset_index().set_index('year')

    # 5년 이동평균
    df_group['MA_5_true'] = df_group[f'{crop}_true'].transform(lambda x: x.shift(1).rolling(5).mean())
    df_group['MA_5_pred'] = df_group[f'{crop}_pred'].transform(lambda x: x.shift(1).rolling(5).mean())

    df_group = df_group.iloc[5:, :] # 5년의 이동평균을 구하면서 처음 5년의 데이터는 사용할 수 없음
    
    # 보험금지급여부
    df_group['보험지급여부_true'] = np.where(df_group['MA_5_true'] >= df_group[f'{crop}_true'], 1, 0)
    df_group['보험지급여부_pred'] = np.where(df_group['MA_5_pred'] >= df_group[f'{crop}_pred'], 1, 0)

    df_group['빈도_mean_true'] = df_group['보험지급여부_true'].expanding().mean()
    df_group['빈도_mean_pred'] = df_group['보험지급여부_pred'].expanding().mean()

    df_group['심도_true'] = np.where(df_group['보험지급여부_true'] == 1, df_group['MA_5_true'] - df_group[f'{crop}_true'], 0)
    df_group['심도_pred'] = np.where(df_group['보험지급여부_pred'] == 1, df_group['MA_5_pred'] - df_group[f'{crop}_pred'], 0)

    df_group['심도_mean_true'] = df_group['심도_true'].expanding().mean()
    df_group['심도_mean_pred'] = df_group['심도_pred'].expanding().mean()

    df_group['보험료_true'] = df_group['빈도_mean_true'] * df_group['심도_mean_true']

    # 검정 연도, 모델의 testset이 2017년_1월 부터이므로 2017년부터 시작
    for target_year in TARGET_YEAR:
        premium = df_group.at[target_year - 1, '보험료_true'] # 전년도까지의 보험료
        damage_true = df_group.at[target_year, '심도_true'] # 해당연도의 손해액(true)
        damage_pred = df_group.at[target_year, '심도_pred'] # 해당연도의 손해액(pred)
        insurance[target_year][crop] = {
            '보험료': premium,
            '손해액_true': damage_true,
            '손해액_pred': damage_pred,
        }
    df_group.to_csv(f'./results/{crop}/premiums_{crop}_v2.csv', encoding='utf-8-sig')

In [5]:
insurance

defaultdict(dict,
            {2017: {'감자': {'보험료': 750.8670420110187,
               '손해액_true': 0.0,
               '손해액_pred': 0.0},
              '고구마': {'보험료': 330.8975179063363,
               '손해액_true': 41.0,
               '손해액_pred': 0.0},
              '고추(화건)': {'보험료': 7695.41252341598,
               '손해액_true': 46844.03283333342,
               '손해액_pred': 74582.53949999996},
              '깻잎': {'보험료': 0.0,
               '손해액_true': 703.8165000000045,
               '손해액_pred': 652.0176083333354},
              '녹두(국산)': {'보험료': 10937.31644628098,
               '손해액_true': 3791.900499999989,
               '손해액_pred': 5367.489499999967},
              '단감': {'보험료': 719.6375426997239,
               '손해액_true': 0.0,
               '손해액_pred': 0.0},
              '당근(무세척)': {'보험료': 619.3675118457309,
               '손해액_true': 0.0,
               '손해액_pred': 0.0},
              '대파': {'보험료': 17.818181685950425,
               '손해액_true': 0.0,
               '손해액_pred': 0

# 연도별 손해율 계산

In [6]:
for year in TARGET_YEAR:
    statics = insurance[year]
    premium_total = 0
    damage_true_total = 0
    damage_pred_total = 0
    for crop in statics.keys():
        premium_total += statics[crop]['보험료']
        damage_true_total += statics[crop]['손해액_true']
        damage_pred_total += statics[crop]['손해액_pred']
    print(f'검정 연도: {year}')
    print(f'\t손해율(true): {damage_true_total / premium_total}')
    print(f'\t손해율(pred): {damage_pred_total / premium_total}')

검정 연도: 2017
	손해율(true): 3.6468893620245466
	손해율(pred): 3.95192792385283
검정 연도: 2018
	손해율(true): 1.8789722338531822
	손해율(pred): 2.074143147314892
검정 연도: 2019
	손해율(true): 0.8262909975006407
	손해율(pred): 0.8442014460815951
검정 연도: 2020
	손해율(true): 0.12289137199500429
	손해율(pred): 0.09315089106356642


In [7]:
test_crop = ['콩', '포도', '양파', '마늘', '고구마', '감자', '고추']
for year in TARGET_YEAR:
    statics = insurance[year]
    premium_total = 0
    damage_true_total = 0
    damage_pred_total = 0
    for crop in test_crop:
        if crop in statics.keys():
            premium_total += statics[crop]['보험료']
            damage_true_total += statics[crop]['손해액_true']
            damage_pred_total += statics[crop]['손해액_pred']
    print(f'검정 연도: {year}')
    print(f'\t손해율(true): {damage_true_total / premium_total}')
    print(f'\t손해율(pred): {damage_pred_total / premium_total}')

검정 연도: 2017
	손해율(true): 0.03230530466727063
	손해율(pred): 0.0
검정 연도: 2018
	손해율(true): 3.4427483686261726
	손해율(pred): 1.9731571348140027
검정 연도: 2019
	손해율(true): 9.159631859520946
	손해율(pred): 8.399442059644146
검정 연도: 2020
	손해율(true): 4.180798934710588
	손해율(pred): 0.0


In [10]:
import pickle

with open('./results/crop_cluster.pickle', 'rb') as f:
    data = pickle.load(f)
data

{'감자': {'crop_class': '식량작물', 'price_class': 0},
 '고구마': {'crop_class': '식량작물', 'price_class': 4},
 '고추(화건)': {'crop_class': '채소', 'price_class': 3},
 '깻잎': {'crop_class': '채소', 'price_class': 2},
 '녹두(국산)': {'crop_class': '식량작물', 'price_class': 3},
 '단감': {'crop_class': '과일', 'price_class': 1},
 '당근(무세척)': {'crop_class': '채소', 'price_class': 0},
 '대파': {'crop_class': '채소', 'price_class': 4},
 '들깨(국산)': {'crop_class': '특용작물', 'price_class': 3},
 '딸기': {'crop_class': '채소', 'price_class': 2},
 '무': {'crop_class': '채소', 'price_class': 0},
 '미나리': {'crop_class': '채소', 'price_class': 0},
 '방울토마토': {'crop_class': '채소', 'price_class': 2},
 '배(신고)': {'crop_class': '과일', 'price_class': 4},
 '배추': {'crop_class': '채소', 'price_class': 0},
 '백참깨(국산)': {'crop_class': '특용작물', 'price_class': 4},
 '복숭아(백도)': {'crop_class': '과일', 'price_class': 1},
 '사과(후지)': {'crop_class': '과일', 'price_class': 4},
 '생강(국산)': {'crop_class': '채소', 'price_class': 4},
 '수박': {'crop_class': '채소', 'price_class': 2},
 '시금치': 

In [38]:
dfs = []

for year in TARGET_YEAR:
    statics = insurance[year]
    premium_total = defaultdict(int)
    damage_true_total = defaultdict(int)
    damage_pred_total = defaultdict(int)
    for crop in statics.keys():
        p_class = data[crop]['price_class']
        
        
        premium_total[p_class] += statics[crop]['보험료']
        damage_true_total[p_class] += statics[crop]['손해액_true']
        damage_pred_total[p_class] += statics[crop]['손해액_pred']
#     print(f'검정 연도: {year}')
#     print(f'cluster_id | 손해율(true) | 손해율(pred)')
#     for p_class in range(7):
#         print(f'{p_class} | {damage_true_total[p_class] / premium_total[p_class]:.4f} | {damage_pred_total[p_class] / premium_total[p_class]:.4f}')
    df = pd.DataFrame({
        '검정연도': [year] * 7,
        'cluster_id': range(7),
        '손해율(true)': [damage_true_total[i] / premium_total[i] for i in range(7)],
        '손해율(pred)': [damage_pred_total[i] / premium_total[i] for i in range(7)],
    })
    dfs.append(df)
    #print(f'\t손해율(pred): {damage_pred_total / premium_total}')
df_total = pd.concat(dfs, axis=0)
df_total

,검정연도,cluster_id,손해율(true),손해율(pred)
0,2017,0,0.473449,1.858337
1,2017,1,4.033635,0.107160
2,2017,2,1.540479,1.581532
3,2017,3,3.954650,4.781895
4,2017,4,3.738867,3.568686
5,2017,5,0.000000,0.438442
6,2017,6,4.276381,3.472149
0,2018,0,2.208625,1.265839
1,2018,1,1.218203,0.000000
2,2018,2,0.000000,1.367975


In [39]:
df_total.to_csv('./results/손해율검정.csv', encoding='utf-8')

In [42]:
with open('./results/cluster_id.pkl', 'rb') as f:
    cluster_id = pickle.load(f)
cluster_id

defaultdict(list,
            {0: ['감자', '당근(무세척)', '무', '미나리', '배추', '양파'],
             4: ['고구마', '대파', '배(신고)', '백참깨(국산)', '사과(후지)', '생강(국산)', '팥(국산)'],
             3: ['고추(화건)', '녹두(국산)', '들깨(국산)', '콩(흰콩국산)'],
             2: ['깻잎', '딸기', '방울토마토', '수박', '애호박', '양배추', '오이(가시)', '토마토'],
             1: ['단감', '복숭아(백도)', '참외', '풋고추'],
             6: ['시금치', '적상추'],
             5: ['포도(캠벨)']})

In [55]:
df_total[df_total['cluster_id'] == 6]

,검정연도,cluster_id,손해율(true),손해율(pred)
6,2017,6,4.276381,3.472149
6,2018,6,0.000000,0.000000
6,2019,6,1.374857,0.000000
6,2020,6,3.119839,11.149880
